In [1]:
from dask_jobqueue import SLURMCluster
from gensim.models import Word2Vec
import pandas as pd
import dask.dataframe as dd
from transformers import BertTokenizer, BertModel
import torch
from torch.utils.dlpack import to_dlpack, from_dlpack
from numba import jit
import numpy as np
from dask import delayed, compute
import gc

# Compose SLURM script
cluster = SLURMCluster(queue='caslake', cores=20, memory='100GB', 
                       processes=20, walltime='03:00:00', interface='ib0',
                       job_extra=['--account=macs30123']
                      )

# Request resources
cluster.scale(jobs=1)

In [3]:
from dask.distributed import Client

client = Client(cluster)
client

<Client: 'tcp://172.25.0.66:35901' processes=10 threads=10, memory=37.30 GiB>

In [4]:
# Patent data scraped from Google Patents
patent_df = pd.read_csv("all_patent_info.csv")
patent_df = patent_df.iloc[250000:]

In [5]:
# Load the BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [ ]:
# Drop rows with null values in the 'abstract' column
patent_df = patent_df.dropna(subset=['abstract'])

# Ensure all entries in the 'abstract' column are strings
patent_df['abstract'] = patent_df['abstract'].astype(str)


ddf = dd.from_pandas(patent_df, npartitions=10)
print("finish to ddf")
# Function to compute BERT embeddings
def compute_bert_embedding(texts):
    inputs = tokenizer(texts, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
    return embeddings

# Define the metadata for the new column as a numpy array
meta = pd.Series(dtype=object)

# Apply the function directly using Dask with metadata
ddf['embeddings'] = ddf['abstract'].apply(lambda x: compute_bert_embedding(x), meta=('embeddings', 'object'))

# Compute the result
result = ddf.compute()

# Ensure embeddings are stored as numpy arrays
result['embeddings'] = result['embeddings'].apply(np.array)

In [ ]:
print(result['embeddings'])

result.to_csv('embeddings_result1.csv', index=False)

In [ ]:
client.close()